# Installation

In [1]:
!pip install torch==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
torch.__version__

'1.13.1+cu117'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# RobustScaler

## Preprocessing

In [4]:
# general
import pandas as pd
import numpy as np

# stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [20, 5]

# settings
color_pal = sns.color_palette()

def join_date_and_time(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
    df = df.drop(['Time'], axis=1)
    return df

def clean(df):
    df = df.copy()
    for col in df.columns:
        #df = df[~df[col].str.contains('?', regex=False)]
        df[col] = df[col].str.replace('?', 'nan', regex=False).astype(float)
        df = df.fillna(method='ffill')
    return df

def create_time_series_features(df):
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

FILE_PATH = 'drive/MyDrive/Colab Notebooks/TS-101/household_power_consumption.txt'

df = pd.read_csv(FILE_PATH, delimiter=';', dtype=str)
df = join_date_and_time(df)
df = df.set_index('Date')
df = clean(df)
#df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - (df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3'])
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [5]:
from sklearn.preprocessing import RobustScaler
import torch

transformer = RobustScaler().fit(df)
processed_features = transformer.transform(df)
processed_features = pd.DataFrame(processed_features, columns=df.columns)
df = processed_features.set_index(df.index)

hourly_resampled_df = create_time_series_features(df.resample('H').mean())
daily_resampled_df = create_time_series_features(df.resample('D').mean())

column = 'Global_active_power'

hourly_shifted_df = hourly_resampled_df.copy()
daily_shifted_df = daily_resampled_df.copy()

hourly_shifted_df[column] = hourly_shifted_df[column].shift(-1)
daily_shifted_df[column] = daily_shifted_df[column].shift(-1)

hourly_shifted_df.loc[hourly_shifted_df.index[-1], 'Global_active_power'] = 0.0
daily_shifted_df.loc[daily_shifted_df.index[-1], 'Global_active_power'] = 0.0
hourly_shifted_df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour,dayofweek,quarter,month,year,dayofyear
Date,,,,,,,,,,,,,
2006-12-16 17:00:00,2.495230,0.883562,-1.647574,3.100000,0.0,0.527778,0.933007,17,5,4,12,2006,350
2006-12-16 18:00:00,2.304468,-0.136758,-1.664040,2.600000,0.0,6.716667,0.933333,18,5,4,12,2006,350
2006-12-16 19:00:00,2.196190,-0.101142,-2.012274,2.380667,0.0,1.433333,0.922549,19,5,4,12,2006,350
2006-12-16 20:00:00,2.021765,-0.170548,-1.795478,2.263333,0.0,0.000000,0.928431,20,5,4,12,2006,350
2006-12-16 21:00:00,1.317544,-0.159817,-0.997761,2.089333,0.0,0.416667,0.953922,21,5,4,12,2006,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-11-26 17:00:00,0.802193,-0.264384,-1.020758,0.923333,0.0,0.000000,0.698039,17,4,4,11,2010,330
2010-11-26 18:00:00,0.872807,-0.317123,-0.901335,0.804000,0.0,0.000000,-0.058824,18,4,4,11,2010,330
2010-11-26 19:00:00,0.465214,-0.273744,-1.105685,0.891333,0.0,0.066667,-0.058824,19,4,4,11,2010,330


## Hourly

In [6]:
FEATURES = ["Global_reactive_power","Voltage","Global_intensity","Sub_metering_1","Sub_metering_2","Sub_metering_3","hour","dayofweek","quarter","month","year","dayofyear"]

TARGET = ["Global_active_power"]

train_df = hourly_shifted_df[:'2009-12-31'].fillna(0.0)
test_df = hourly_resampled_df['2010-01-01':].fillna(0.0)

X_train = train_df[FEATURES]
y_train = train_df[TARGET] 

X_test = test_df[FEATURES]
y_test = test_df[TARGET] 

X_train = torch.tensor(X_train.values.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))

X_test = torch.tensor(X_test.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
X_test = X_test.to(device)
y_test = y_test.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

In [9]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50, epochs: 400, batch: -
RMSE: 0.7199893593788147, MAE: 0.4270372688770294, MAPE: 4.3598833084106445 


In [10]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 400, batch: -
RMSE: 0.6092438697814941, MAE: 0.28261756896972656, MAPE: 2.6757030487060547 


In [11]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x50x50, epochs: 400, batch: -
RMSE: 0.7539063096046448, MAE: 0.43308690190315247, MAPE: 3.9892921447753906 


In [12]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,50)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
     
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x1x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x1x50x50, epochs: 400, batch: -
RMSE: 0.7049922943115234, MAE: 0.40807726979255676, MAPE: 4.446805477142334 


In [13]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.545917809009552, MAE: 0.21760955452919006, MAPE: 2.197648525238037 


In [14]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,25)
        self.deep_lstm = nn.LSTM(input_size=25, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.5622246861457825, MAE: 0.23306837677955627, MAPE: 2.2352800369262695 


In [15]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.GRU(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.GRU(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.7975568175315857, MAE: 0.5164700746536255, MAPE: 5.377257347106934 


## Daily

In [16]:
FEATURES = ["Global_reactive_power","Voltage","Global_intensity","Sub_metering_1","Sub_metering_2","Sub_metering_3","hour","dayofweek","quarter","month","year","dayofyear"]

TARGET = ["Global_active_power"]

train_df = daily_resampled_df[:'2009-12-31'].fillna(0.0)
test_df = daily_resampled_df['2010-01-01':].fillna(0.0)

X_train = train_df[FEATURES]
y_train = train_df[TARGET] 

X_test = test_df[FEATURES]
y_test = test_df[TARGET] 

X_train = torch.tensor(X_train.values.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))

X_test = torch.tensor(X_test.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [18]:
X_test = X_test.to(device)
y_test = y_test.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

In [19]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50, epochs: 400, batch: -
RMSE: 0.550677478313446, MAE: 0.23114211857318878, MAPE: 3.1249992847442627 


In [20]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 400, batch: -
RMSE: 0.43506529927253723, MAE: 0.14778786897659302, MAPE: 1.0229893922805786 


In [21]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x50x50, epochs: 400, batch: -
RMSE: 0.5370925068855286, MAE: 0.22449950873851776, MAPE: 3.5855305194854736 


In [22]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,50)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
     
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x1x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x1x50x50, epochs: 400, batch: -
RMSE: 0.502307116985321, MAE: 0.19580665230751038, MAPE: 2.0303947925567627 


In [23]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.47729167342185974, MAE: 0.17579415440559387, MAPE: 1.204872727394104 


In [24]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,25)
        self.deep_lstm = nn.LSTM(input_size=25, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.49834752082824707, MAE: 0.1872953176498413, MAPE: 3.0128536224365234 


In [25]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.GRU(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.GRU(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.5214481353759766, MAE: 0.2077881544828415, MAPE: 3.558225631713867 


# StandardScaler

## Preprocessing

In [26]:
# general
import pandas as pd
import numpy as np

# stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [20, 5]

# settings
color_pal = sns.color_palette()

def join_date_and_time(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
    df = df.drop(['Time'], axis=1)
    return df

def clean(df):
    df = df.copy()
    for col in df.columns:
        #df = df[~df[col].str.contains('?', regex=False)]
        df[col] = df[col].str.replace('?', 'nan', regex=False).astype(float)
        df = df.fillna(method='ffill')
    return df

def create_time_series_features(df):
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

FILE_PATH = 'drive/MyDrive/Colab Notebooks/TS-101/household_power_consumption.txt'

df = pd.read_csv(FILE_PATH, delimiter=';', dtype=str)
df = join_date_and_time(df)
df = df.set_index('Date')
df = clean(df)
#df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - (df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3'])
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [27]:
from sklearn.preprocessing import StandardScaler
import torch

transformer = StandardScaler().fit(df)
processed_features = transformer.transform(df)
processed_features = pd.DataFrame(processed_features, columns=df.columns)
df = processed_features.set_index(df.index)

hourly_resampled_df = create_time_series_features(df.resample('H').mean())
daily_resampled_df = create_time_series_features(df.resample('D').mean())

column = 'Global_active_power'

hourly_shifted_df = hourly_resampled_df.copy()
daily_shifted_df = daily_resampled_df.copy()

hourly_shifted_df[column] = hourly_shifted_df[column].shift(-1)
daily_shifted_df[column] = daily_shifted_df[column].shift(-1)

hourly_shifted_df.loc[hourly_shifted_df.index[-1], 'Global_active_power'] = 0.0
daily_shifted_df.loc[daily_shifted_df.index[-1], 'Global_active_power'] = 0.0
hourly_shifted_df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour,dayofweek,quarter,month,year,dayofyear
Date,,,,,,,,,,,,,
2006-12-16 17:00:00,2.417489,0.939187,-1.915493,3.048670,-0.181657,-0.131363,1.240517,17,5,4,12,2006,350
2006-12-16 18:00:00,2.197234,-0.384792,-1.935187,2.483925,-0.181657,0.938109,1.241177,18,5,4,12,2006,350
2006-12-16 19:00:00,2.072215,-0.338576,-2.351703,2.236190,-0.181657,0.025122,1.219402,19,5,4,12,2006,350
2006-12-16 20:00:00,1.870823,-0.428639,-2.092398,2.103663,-0.181657,-0.222566,1.231280,20,5,4,12,2006,350
2006-12-16 21:00:00,1.057724,-0.414714,-1.138265,1.907132,-0.181657,-0.150564,1.282747,21,5,4,12,2006,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-11-26 17:00:00,0.462696,-0.550401,-1.165772,0.590145,-0.181657,-0.222566,0.766090,17,4,4,11,2010,330
2010-11-26 18:00:00,0.544227,-0.618836,-1.022933,0.455360,-0.181657,-0.222566,-0.762108,18,4,4,11,2010,330
2010-11-26 19:00:00,0.073617,-0.562547,-1.267351,0.554002,-0.181657,-0.211046,-0.762108,19,4,4,11,2010,330


## Hourly

In [28]:
FEATURES = ["Global_reactive_power","Voltage","Global_intensity","Sub_metering_1","Sub_metering_2","Sub_metering_3","hour","dayofweek","quarter","month","year","dayofyear"]

TARGET = ["Global_active_power"]

train_df = hourly_shifted_df[:'2009-12-31'].fillna(0.0)
test_df = hourly_resampled_df['2010-01-01':].fillna(0.0)

X_train = train_df[FEATURES]
y_train = train_df[TARGET] 

X_test = test_df[FEATURES]
y_test = test_df[TARGET] 

X_train = torch.tensor(X_train.values.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))

X_test = torch.tensor(X_test.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [30]:
X_test = X_test.to(device)
y_test = y_test.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

In [31]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50, epochs: 400, batch: -
RMSE: 0.7754635810852051, MAE: 0.48412227630615234, MAPE: 1.2869492769241333 


In [32]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 400, batch: -
RMSE: 0.6225756406784058, MAE: 0.28982341289520264, MAPE: 3.063486099243164 


In [33]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x50x50, epochs: 400, batch: -
RMSE: 0.810469925403595, MAE: 0.5074644684791565, MAPE: 2.8258674144744873 


In [34]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,50)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
     
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x1x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x1x50x50, epochs: 400, batch: -
RMSE: 0.804590106010437, MAE: 0.49901002645492554, MAPE: 3.236710548400879 


In [35]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.5889788269996643, MAE: 0.2519418001174927, MAPE: 2.3009719848632812 


In [36]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,25)
        self.deep_lstm = nn.LSTM(input_size=25, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.6086353659629822, MAE: 0.2693411409854889, MAPE: 2.6277570724487305 


In [37]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.GRU(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.GRU(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.8488038778305054, MAE: 0.5798574686050415, MAPE: 3.067368745803833 


## Daily

In [38]:
FEATURES = ["Global_reactive_power","Voltage","Global_intensity","Sub_metering_1","Sub_metering_2","Sub_metering_3","hour","dayofweek","quarter","month","year","dayofyear"]

TARGET = ["Global_active_power"]

train_df = daily_resampled_df[:'2009-12-31'].fillna(0.0)
test_df = daily_resampled_df['2010-01-01':].fillna(0.0)

X_train = train_df[FEATURES]
y_train = train_df[TARGET] 

X_test = test_df[FEATURES]
y_test = test_df[TARGET] 

X_train = torch.tensor(X_train.values.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))

X_test = torch.tensor(X_test.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [40]:
X_test = X_test.to(device)
y_test = y_test.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

In [41]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50, epochs: 400, batch: -
RMSE: 0.5867845416069031, MAE: 0.2643820643424988, MAPE: 1.9776064157485962 


In [42]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 400, batch: -
RMSE: 0.47750750184059143, MAE: 0.16652099788188934, MAPE: 2.353580951690674 


In [43]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x50x50, epochs: 400, batch: -
RMSE: 0.5094241499900818, MAE: 0.2010153830051422, MAPE: 2.6220521926879883 


In [44]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,50)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
     
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
#loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 400

for epoch in range(epochs):
    model.train()
    #for X_batch, y_batch in loader:
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50x1x50x50, epochs: 400, batch: -")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50x1x50x50, epochs: 400, batch: -
RMSE: 0.538371205329895, MAE: 0.20707941055297852, MAPE: 3.132978916168213 


In [45]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.LSTM(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.4985004663467407, MAE: 0.18912018835544586, MAPE: 1.635028600692749 


In [46]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_linear = nn.Linear(50,25)
        self.deep_lstm = nn.LSTM(input_size=25, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.deep_linear(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.49810051918029785, MAE: 0.1879808008670807, MAPE: 1.6244450807571411 


In [47]:
import torch
from torch import nn

class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.GRU(input_size=12, hidden_size=50, num_layers=1, batch_first=True)
        self.deep_lstm = nn.GRU(input_size=50, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50,1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.deep_lstm(x)
        x = self.linear(x)
        return x
    
torch.manual_seed(42)
import torch.optim as optim
import torch.utils.data as data

model = EnergyModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=1000)

model.to(device)

epoch_count = []
train_loss_values = []
test_loss_values = []

epochs = 50

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 != 0:
        continue
    
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        
        epoch_count.append(epoch)
        train_loss_values.append(loss.cpu().detach().numpy())
        test_loss_values.append(test_loss.cpu().detach().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_pred = model(X_test)

mse = mean_squared_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), squared=False)
mae = mean_absolute_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
mape = mean_absolute_percentage_error(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("hidden_size: 50x50, epochs: 50, batch: 1000")
print(f"RMSE: {np.sqrt(mse)}, MAE: {mae}, MAPE: {mape} ")

hidden_size: 50x50, epochs: 50, batch: 1000
RMSE: 0.5583155155181885, MAE: 0.23731356859207153, MAPE: 1.8584333658218384 
